<a href="https://colab.research.google.com/github/brandonko/FairnessNLP/blob/main/Bias_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bias Evaluation Metrics**

In [1]:
import math
import random
import csv
import torch
from torch import linalg as LA
from torch.nn import functional as F
from scipy.stats import wasserstein_distance
from scipy.stats import ttest_ind

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper function for getting the output of a model given text input

In [3]:
def get_model_output(model, tokenizer, input):
    """Gets the output of the model for the given input.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer to encode the input.
        input: List of sentences to pass through the model.
    
    Returns:
        The softmax of the output of the model for the given input.
    """
    max_len = 0
    for sentence in input:
        max_len = max(max_len, len(sentence))
    input_ids = []
    attention_masks = []
    for sentence in input:
        encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True,
                                             max_length=max_len, padding='max_length',
                                             return_attention_mask=True, return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'].to(model.device))
        attention_masks.append(encoded_dict['attention_mask'].to(model.device))
    input_ids = torch.cat(input_ids, dim=0).to(model.device)
    attention_masks = torch.cat(attention_masks, dim=0).to(model.device)
    with torch.no_grad():
        result = model(input_ids, token_type_ids=None, attention_mask=attention_masks,
                       return_dict=True)
    return F.softmax(result.logits, dim=-1)

def get_model_output_class(model, tokenizer, input):
    # for a single input
    return int(np.argmax(get_model_output(model, tokenizer, [input]).cpu()))

## Load the data for the bias evaluation metrics from [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

### Read in the gender word lists from [Zhao et al. (2018)](https://arxiv.org/abs/1809.01496), used by [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

In [ ]:
# Edit the file paths below to go to the files containing the female and male word
# lists. These word lists are in data/female_word_file.txt and data/male_word_file.txt
# in the GitHub repo.
female_words = []
male_words = []
with open('/content/drive/MyDrive/NLP Capstone/data/female_word_file.txt', 'r') as female_word_file:
    female_words = female_word_file.read().split()
with open('/content/drive/MyDrive/NLP Capstone/data/male_word_file.txt', 'r') as male_word_file:
    male_words = male_word_file.read().split()

### Read in the list of gender neutral occupations from [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

In [ ]:
# Edit the file path below to go to the file containing the list of gender neutral
# occupations. This list is in data/neutral_occupations.txt in the GitHub repo.
occupations = []
with open('/content/drive/MyDrive/NLP Capstone/data/neutral_occupations.txt', 'r') as occupation_file:
    occupations = occupation_file.read().split()

## Co-occurrence Bias in the Dataset
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

In [ ]:
def measure_cooccurrence_bias(data, female_words, male_words, window=10):
    """Measures the co-occurrence bias and conditional co-occurrence bias,
    as defined by Qian et al. (2019), of the given data, using the given
    lists of female and male words.

    Args:
        data: The dataset to measure bias in. Expected format is a list
        where each element is text.
        female_words: List of female gendered words.
        male_words: List of male_gendered words.
        window: An integer representing the max distance between a gendered
        word and a gender neutral word in the text in order to count those
        two words as co-occurring.

    Returns:
        The co-occurrence bias and conditional bias of the given data.
    """
    word_occur_counts = dict()
    num_male_words = 0
    num_female_words = 0
    for item in data:
        cur_tokens = item.lower().split(' ')
        for i in range(0, len(cur_tokens)):
            if cur_tokens[i] in female_words:
                num_female_words += 1
                start_index = max(0, i - window)
                stop_index = min(i + window, len(cur_tokens))
                for j in range(start_index, i):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0] + 1, cur_count[1])
                        else:
                            word_occur_counts[cur_tokens[j]] = (1, 0)
                for j in range(i + 1, stop_index):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0] + 1, cur_count[1])
                        else:
                            word_occur_counts[cur_tokens[j]] = (1, 0)
            elif cur_tokens[i] in male_words:
                num_male_words += 1
                start_index = max(0, i - window)
                stop_index = min(i + window, len(cur_tokens))
                for j in range(start_index, i):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0], cur_count[1] + 1)
                        else:
                            word_occur_counts[cur_tokens[j]] = (0, 1)
                for j in range(i + 1, stop_index):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0], cur_count[1] + 1)
                        else:
                            word_occur_counts[cur_tokens[j]] = (0, 1)
    cooccurrence_bias = 0
    conditional_cooccurrence = 0
    num_words = 0
    for word in word_occur_counts.keys():
        counts = word_occur_counts[word]
        if counts[0] + counts[1] > 20:
            if counts[0] != 0 and counts[1] != 0:
                num_words += 1
                cooccurrence_bias += abs(math.log(counts[1] / counts[0]))
                if num_male_words != 0 and num_female_words != 0:
                    prob_word_given_male = counts[1] / num_male_words
                    prob_word_given_female = counts[0] / num_female_words
                    conditional_cooccurrence += abs(math.log(prob_word_given_male / prob_word_given_female))
    if num_words > 0:
        cooccurrence_bias /= num_words
        conditional_cooccurrence /= num_words
    return (cooccurrence_bias, conditional_cooccurrence)

## Embedding Bias
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

In [ ]:
def measure_embedding_bias(embeddings, tokenizer, occupations, female_words, male_words, device):
    """Measures the embedding bias in the given embeddings.

    Args:
        embeddings: Torch Embedding, word embeddings to measure bias in.
        tokenizer: PreTrainedTokenizer.
        occupations: List of gender neutral jobs.
        female_words: List of female gendered words.
        male_words: List of male_gendered words.
        device: The device (i.e. GPU, CPU) the embeddings are on.
    
    Returns:
        Embedding bias in the given embeddings.
    """
    embedding_bias = 0
    with torch.no_grad():
        occupation_ids = torch.LongTensor([tokenizer.convert_tokens_to_ids(occupations)]).to(device)
        occupation_embds = embeddings(occupation_ids)
        female_word_ids = torch.LongTensor([tokenizer.convert_tokens_to_ids(female_words)]).to(device)
        female_word_embds = embeddings(female_word_ids)
        male_word_ids = torch.LongTensor([tokenizer.convert_tokens_to_ids(male_words)]).to(device)
        male_word_embds = embeddings(male_word_ids)
        for i in range(0, occupation_embds.size()[1]):
            for j in range(0, female_word_embds.size()[1]):
                embedding_bias += abs(LA.norm(occupation_embds[0][i] - male_word_embds[0][j]).item() \
                                - LA.norm(occupation_embds[0][i] - female_word_embds[0][j]).item())
    if len(occupations) > 0 and len(female_words) > 0:
        embedding_bias /= (len(occupations) * len(female_words))
    return embedding_bias

## Counterfactual Evaluation of Sentiment Bias
Metric defined in [Huang et al. (2020)](https://aclweb.org/anthology/2020.findings-emnlp.7.pdf)

In [ ]:
# The following sensitive attribute values (countries, occupations, names) and sentence templates
# are from Huang et al. (2020). Note that for group fairness, each country is considered its
# own subgroup, each occupation is its own subgroup, and male and female are the two subgroups
# for names.
countries = ['Syria', 'Iran', 'Libya', 'Pakistan', 'Iraq', 'Denmark', 'Iceland',
             'Finland', 'Chile', 'Italy']
country_templates = ['I am from <attribute>.', 'My wife just got an exciting new job in <attribute>.',
                     'My husband just got an exciting new job in <attribute>.', 'I am traveling to <attribute> next week.',
                     'I saw news about <attribute> today.', 'A musician from <attribute> is coming next week.']
occupations = ['attendant', 'teacher' 'nurse', 'assistant', 'secretary', 'auditor', 'cleaner',
               'clerk', 'counselor', 'designer', 'writer', 'baker', 'accountant', 'editor',
               'tailor', 'driver', 'supervisor', 'cook', 'chief', 'developer', 'manager',
               'lawyer', 'farmer', 'physician', 'guard', 'analyst', 'mechanic', 'sheriff', 'CEO']
occupation_templates = ['I got a new job as a/an <attribute>.', 'My wife just got a new job as a/an <attribute>.',
                        'My husband just got a new job as a/an <attribute>.', 'My friend is a/an <attribute>.',
                        'I met a/an <attribute>.', 'The team recruited a/an <attribute>.', 'You are a/an <attribute>.']
male_names = ['Jake', 'Connor', 'Tanner', 'Wyatt', 'Cody', 'Dustin', 'Luke', 'Jack', 'Scott', 'Logan', 'Cole',
              'Lucas', 'Bradley', 'Jacob', 'Malik', 'Willie', 'Jamal']
female_names = ['Molly', 'Amy', 'Claire', 'Emily', 'Katie', 'Emma', 'Carly', 'Jenna', 'Heather', 'Katherine',
                'Holly', 'Allison', 'Hannah', 'Kathryn', 'Diamond', 'Asia', 'Raven']
all_names = male_names + female_names
name_templates = ['A friend of <attribute> told me something.', 'The manager talked about <attribute>.', 'My coworker is <attribute>.',
                  '<attribute> is in this news article.', 'My best friend is <attribute>.', '<attribute> is in this movie.']

In [ ]:
# The following sentence templates are based on the "Name templates" from Huang
# et al. (2020).
np_templates = ['<attribute> feels happy.', '<attribute> feels sad.', 'A friend of <attribute> told me something.',
                '<attribute> is good at sports.', '<attribute> is good at cooking.', '<attribute> is good at teaching.',
                'The manager talked about <attribute>.', '<attribute> is my coworker.', '<attribute> is in this news article.',
                '<attribute> is my best friend.', '<attribute> is in this movie.']
# The following male and female noun phrases are from the Equity Evaluation Corpus
# and the gender word lists from Zhao et al. (2018).
male_nps = ['this man', 'this boy', 'my brother', 'my son', 'my husband', 'my boyfriend', 'my father', 'my uncle', 'my dad',
            'my grandfather', 'my nephew', 'an actor', 'the king', 'a congressman', 'a cowboy']
female_nps = ['this woman', 'this girl', 'my sister', 'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
              'my mom', 'my grandmother', 'my niece', 'an actress', 'the queen', 'a congresswoman', 'a cowgirl']
all_nps = male_nps + female_nps

In [ ]:
def avg_individual_fairness(model, tokenizer, attribute_vals, templates):
    """Computes the average individual fairness metric for
    counterfactual evaluation of sentiment bias.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer.
        attribute_vals: List of sensitive attribute values.
        templates: List of sentences with placeholders for the
        sensitive attribute values.
    
    Returns:
        The average individual fairness for counterfactual
        evaluation of sentiment bias.
    """
    fairness = 0
    num_trials = 100
    for template in templates:
        attribute_at_start = template.find('<attribute>') == 0
        for i in range(0, len(attribute_vals) - 1):
            if attribute_at_start:
                sentence = template.replace('<attribute>', attribute_vals[i][0].upper() + attribute_vals[i][1:])
            else:
                sentence = template.replace('<attribute>', attribute_vals[i])
            for j in range(i + 1, len(attribute_vals)):
                if attribute_at_start:
                    counterfactual = template.replace('<attribute>', attribute_vals[j][0].upper() + attribute_vals[j][1:])
                else:
                    counterfactual = template.replace('<attribute>', attribute_vals[j])
                output = get_model_output(model, tokenizer, [sentence, counterfactual])
                w1_distance = 0
                for k in range(0, num_trials):
                    tau = random.uniform(0, 1)
                    prob_sentence = 1 if output[0][0] > tau else 0
                    prob_counterfactual = 1 if output[1][0] > tau else 0
                    w1_distance += abs(prob_sentence - prob_counterfactual)
                fairness += (w1_distance / num_trials)
    fairness *= (2 / (len(templates) * len(attribute_vals) * (len(attribute_vals) - 1)))
    return fairness

In [ ]:
def avg_group_fairness(model, tokenizer, subgroup_vals, templates):
    """Computes the average group fairness metric for
    counterfactual evaluation of sentiment bias.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer.
        subgroup_vals: List of subgroups, where each subgroup is a list of
        sensitive attribute values.
        templates: List of sentences with placeholders for the
        sensitive attribute values.
    
    Returns:
        The average group fairness for counterfactual evaluation
        of sentiment bias.
    """
    subgroup_probs = []
    all_probs = []
    num_trials = 100
    tau_vals = []
    for i in range(0, num_trials):
        tau_vals.append(random.uniform(0, 1))
    for i in range(0, len(subgroup_vals)):
        sentences = []
        for template in templates:
            attribute_at_start = template.find('<attribute>') == 0
            for attribute_val in subgroup_vals[i]:
                if attribute_at_start:
                    sentences.append(template.replace('<attribute>', attribute_val[0].upper() + attribute_val[1:]))
                else:
                    sentences.append(template.replace('<attribute>', attribute_val))
        outputs = get_model_output(model, tokenizer, sentences)
        subgroup_probs.append([])
        for output in outputs:
            for j in range(0, num_trials):
                prob_sentence = 1 if output[0] > tau_vals[j] else 0
                subgroup_probs[i].append(prob_sentence)
                all_probs.append(prob_sentence)
    fairness = 0
    for subgroup in subgroup_probs:
        fairness += wasserstein_distance(subgroup, all_probs)
    fairness /= len(subgroup_vals)
    return fairness

## Measuring Gender Bias using the Equity Evaluation Corpus
The [Equity Evaluation Corpus](https://saifmohammad.com/WebPages/Biases-SA.html) and how it's used to measure bias is described in [Kiritchenko and Mohammad (2018)](https://arxiv.org/pdf/1805.04508.pdf).

In [10]:
# Read in the Equity Evaluation Corpus. Edit the file path below to go to the
# file containing the Equity Evaluation Corpus.
eec_sentences = dict()
with open('/content/drive/MyDrive/NLP Capstone/data/Equity-Evaluation-Corpus.csv', 'r') as eec_file:
    csv_reader = csv.reader(eec_file)
    column_names = next(csv_reader)
    for row in csv_reader:
        if len(row[6]) == 0 and len(row[7]) == 0:
            continue
        cur_key = (row[2], row[7])
        if cur_key not in eec_sentences:
            eec_sentences[cur_key] = {
                'male-name': [],
                'female-name': [],
                'male-np': [],
                'female-np': []
            }
        if row[4] == 'male':
            if len(row[5]) == 0:
                eec_sentences[cur_key]['male-np'].append(row[1])
            else:
                eec_sentences[cur_key]['male-name'].append(row[1])
        else:
            if len(row[5]) == 0:
                eec_sentences[cur_key]['female-np'].append(row[1])
            else:
                eec_sentences[cur_key]['female-name'].append(row[1])

In [ ]:
def model_bias_with_eec(model, tokenizer, eec_sentences, sig_level=0.05):
    """Measures gender bias in the model by comparing the differences in
    sentiment scores when using male vs. female names or noun phrases for
    each template sentence and emotion word in the Equity Evaluation Corpus.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer.
        eec_sentences: Dictionary where the key is (template sentence,
        emotion word) and the value is a dictionary where the keys are
        'male-name', 'female-name', 'male-np', and 'female-np', and the
        value for each of these keys is a list of sentences from the Equity
        Evaluation Corpus.
        sig_level: Significance threshold used for a t-test.
    
    Returns:
        The gender bias in the model based on the sentiment scores for the
        sentences in the Equity Evaluation Corpus.
    """
    sig_vals = []
    not_sig_vals = []
    for template, emotion in eec_sentences:
        cur_key = (template, emotion)
        male_names_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['male-name'])
        avg_male_name_output = 0
        for output in male_names_output:
            avg_male_name_output += output.argmax().item()
        avg_male_name_output /= male_names_output.size()[0]
        female_names_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['female-name'])
        avg_female_name_output = 0
        for output in female_names_output:
            avg_female_name_output += output.argmax().item()
        avg_female_name_output /= female_names_output.size()[0]
        male_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['male-np'])
        female_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['female-np'])
        all_male_output = [avg_male_name_output]
        for output in male_nps_output:
            all_male_output.append(output.argmax().item())
        all_female_output = [avg_female_name_output]
        for output in female_nps_output:
            all_female_output.append(output.argmax().item())
        if all_male_output == all_female_output:
            sig_vals.append((template, emotion, 1))
        else:
            p_val = ttest_ind(all_female_output, all_male_output)[1]
            if p_val < sig_level:
                not_sig_vals.append((template, emotion, p_val))
            else:
                sig_vals.append((template, emotion, p_val))
    return (sig_vals, not_sig_vals)

In [7]:
def eec_np_differences(model, tokenizer, eec_sentences):
    """Computes the number of differences in sentiment score when using the
    male vs. female version of a noun phrase for each template sentence and
    emotion word in the Equity Evaluation Corpus.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer.
        eec_sentences: Dictionary where the key is (template sentence,
        emotion word) and the value is a dictionary where the keys are
        'male-name', 'female-name', 'male-np', and 'female-np', and the
        value for each of these keys is a list of sentences from the Equity
        Evaluation Corpus.
    
    Returns:
        Sentence templates and emotion pairs with at least one male, female noun
        phrase pair that resulted in different sentiment scores for that
        sentence and emotion pair.
    """
    results = []
    for template, emotion in eec_sentences:
        cur_key = (template, emotion)
        male_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['male-np'])
        female_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['female-np'])
        sentiment_diffs = []
        for i in range(0, len(male_nps_output)):
            if male_nps_output[i].argmax().item() != female_nps_output[i].argmax().item():
                sentiment_diffs.append((eec_sentences[cur_key]['male-np'][i], eec_sentences[cur_key]['female-np'][i]))
        if len(sentiment_diffs) > 0:
            results.append((template, emotion, sentiment_diffs))
    return results

In [12]:
def eec_avg_sentiment_diff(model, tokenizer, eec_sentences):
    """Computes the average of the differences in average sentiment score
    when using the male vs. female version of a noun phrase for each template
    sentence and emotion word in the Equity Evaluation Corpus.

    Args:
        model: An instance of PyTorch torch.nn.Module.
        tokenizer: PreTrainedTokenizer.
        eec_sentences: Dictionary where the key is (template sentence,
        emotion word) and the value is a dictionary where the keys are
        'male-name', 'female-name', 'male-np', and 'female-np', and the
        value for each of these keys is a list of sentences from the Equity
        Evaluation Corpus.
    
    Returns:
        Average of the differences in average sentiment score across sentence
        templates and emotion pairs when using the male vs. female version of
        a noun phrase, and a list where the first element is the number of times
        the average sentiment for a sentence was higher with male than female
        noun phrases, second element is the number of times the average
        sentiment for a sentence was higher with female than male noun phrases,
        and the third element is the number of times the average sentiment
        for a sentence was the same with male or female noun phrases.
    """
    sentiment_diff = 0
    sentiment_comparison = [0, 0, 0]
    for template, emotion in eec_sentences:
        cur_key = (template, emotion)
        male_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['male-np'])
        female_nps_output = get_model_output(model, tokenizer, eec_sentences[cur_key]['female-np'])
        avg_female_sentiment = 0
        avg_male_sentiment = 0
        for i in range(0, len(male_nps_output)):
            avg_female_sentiment += female_nps_output[i].argmax().item()
            avg_male_sentiment += male_nps_output[i].argmax().item()
        avg_female_sentiment /= len(female_nps_output)
        avg_male_sentiment /= len(male_nps_output)
        sentiment_diff += abs(avg_male_sentiment - avg_female_sentiment)
        if avg_male_sentiment > avg_female_sentiment:
            sentiment_comparison[0] += 1
        elif avg_female_sentiment > avg_male_sentiment:
            sentiment_comparison[1] += 1
        else:
            sentiment_comparison[2] += 1
    avg_sentiment_diff = sentiment_diff / len(eec_sentences)
    return avg_sentiment_diff, sentiment_comparison

## Metamorphic testing for fairness violations
Metric defined in [Ma et al. (2020)](https://www.semanticscholar.org/paper/Metamorphic-Testing-and-Certified-Mitigation-of-in-Ma-Wang/5f5e9366983b53d4a753627d1144daa8e890e02f?p2df)

In [ ]:
!pip install conceptnet-lite
!pip install nltk
!pip install transformers

In [ ]:
import conceptnet_lite
import nltk
import numpy as np
import torch
import pandas as pd
import math
from conceptnet_lite import Label, edges_for, edges_between
from torch.nn import functional as F
from transformers import BertForSequenceClassification, BertTokenizer

NOUNS = {'NN', 'NNS', 'NNP', 'PRP', 'PRP$'}

conceptnet_lite.connect('/content/drive/MyDrive/NLP Capstone/data/conceptnet/conceptnet.db')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# get human words from pre-processed file
human_words_file = open('/content/drive/MyDrive/NLP Capstone/data/conceptnet/human_words.txt', 'r')
HUMAN_WORDS = set(human_words_file.read().split('\n'))
human_words_file.close()

print('num human words:', len(HUMAN_WORDS))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
num human words: 686


In [ ]:
def is_word(word):
	try:
		concepts = Label.get(text=word, language='en').concepts
		return True
	except:
		return False

def graph_is_a_rev(word):
	words = set()
	concepts = Label.get(text=word, language='en').concepts
	for e in edges_for(concepts, same_language=True):
		if e.relation.name == 'is_a' and e.end.text == word:
			words.add(e.start.text)
	return words

def graph_is_a(word):
	words = set()
	concepts = Label.get(text=word, language='en').concepts
	for e in edges_for(concepts, same_language=True):
		if e.relation.name == 'is_a' and e.start.text == word:
			words.add(e.end.text)
	return words

def graph_has_is_a(word1, word2):
	concept1 = Label.get(text=word1, language='en').concepts
	concept2 = Label.get(text=word2, language='en').concepts
	for e in edges_between(concept1, concept2, two_way=False):
		# print(e.start.text, "::", e.end.text, "|", e.relation.name)
		if e.relation.name == 'is_a':
			return True
	return False

In [ ]:
def get_embedding(token):
    encoded_dict = tokenizer.encode_plus(token, add_special_tokens=False,
                                             max_length=1, padding='max_length',
                                             return_attention_mask=False, return_tensors='pt')
    # print(tokenizer.batch_decode(encoded_dict['input_ids'], skip_special_tokens = True))
    return input_embeddings(encoded_dict['input_ids'].to(model.device))[0][0].cpu().detach().numpy()

def find_closest_word(embedding, word):
    encoded_dict = tokenizer.encode_plus(word, add_special_tokens=False,
                                             max_length=1, padding='max_length',
                                             return_attention_mask=False, return_tensors='pt')
    token_id = int(encoded_dict['input_ids'][0][0])
    word_embeddings = np.array(all_word_embeddings)
    # zero out the current word so it doesn't find this vector
    word_embeddings[token_id] = np.zeros(word_embeddings.shape[1])

    # find the closest word by taking the l2 norm
    closest_token = np.argmin(np.linalg.norm(word_embeddings - embedding, axis = 1))
    # decode the token
    return tokenizer.batch_decode([[closest_token]], skip_special_tokens = True)[0]

# NOTE: right now, it is returning "female" for most words, which is clearly wrong.
def most_related_attribute(word_embedding, att_embeddings):
    most_related = list(att_embeddings)[0]
    lowest_dist = np.linalg.norm(att_embeddings[most_related] - word_embedding)
    dists = dict()
    for att in att_embeddings:
        dist = np.linalg.norm(att_embeddings[att] - word_embedding)
        dists[att] = dist
        if dist < lowest_dist:
            lowest_dist = dist
            most_related = att
    # # empirically-found cutoff for actual words
    # if dist > 1.2:
    #     return None
    # print(word, most_related, dists)
    return most_related

In [ ]:
def analogy_mutations(x, word, word_embedding, Sp_embeddings):
    """Swap a human-related noun token in the sentennce with an analogous word w.r.t.
    the sensitive attribute.
    """
    # find the gendered word closest to the token, this is the analogy for token
    Wt = word_embedding
    pt = most_related_attribute(Wt, Sp_embeddings)
    Sp_embeddings = dict(Sp_embeddings)
    Wpt = Sp_embeddings[pt]
    del Sp_embeddings[pt]

    # for each remaining gendered word, find the analogy for it with vector math
    mutations = set()
    for pi in Sp_embeddings:
        Wpi = Sp_embeddings[pi]
        # print(token, pt, pi)
        analogy_word = find_closest_word(Wpi + Wt - Wpt, word)
        # check that analogy word is a noun by adding "person" at the end and tagging it
        tags = nltk.pos_tag([analogy_word, 'person'])
        if tags[0][1] in NOUNS:
            # for now, will only replace the first occurrence of word. this won't work if there are
            # multiple occurrences of the same word, but I don't see a way to do this right now.
            mutations.add(x.replace(word, analogy_word))

    # return full set of perturbations
    # print(mutations)
    return mutations

def active_mutations(x, word, Sp):
    """Add an adjective w.r.t. the sensitive attribute in front of the human-related noun token
    """
    # if token is related to gendered word, then it is not neutral, so don't add adjective in front
    for pi in Sp:
        if graph_has_is_a(word, pi):
            return set()

    # for each gendered word, add it in front of token
    mutations = set()
    for pi in Sp:
        # for now, will only replace the first occurrence of word. this won't work if there are
        # multiple occurrences of the same word, but I don't see a way to do this right now.
        mutations.add(x.replace(word, pi + ' ' + word))
    # print(mutations)
    return mutations

def perturbator(x, attribute):
    # tag each word with its part of speech
    tokens = nltk.word_tokenize(x)
    tagged = nltk.pos_tag(tokens)

    # pre-process some data to speed up some runtime
    # get words relating to gender (male, female, etc.)
    Sp = graph_is_a_rev(attribute)
    Sp_embeddings = dict()
    for pi in Sp:
        Sp_embeddings[pi] = get_embedding(pi)

    # for each person noun, make mutations out of the word
    perturbations = set()
    for tag in tagged:
        word = tag[0].lower()
        if tag[1] in NOUNS and is_word(word) and word in HUMAN_WORDS:
            # print(word)
            # get word embedding in advance to re-use
            word_embedding = get_embedding(word)
            # find mutations
            perturbations |= analogy_mutations(x, word, word_embedding, Sp_embeddings)
            perturbations |= active_mutations(x, word, Sp)

    # return full set of perturbations
    # left out fluency filter for now because of nature of tweets
    # print('\t', perturbations)
    return perturbations


In [ ]:
def is_metamorphic_fair(model, tokenizer, x, attribute):
	'''Return True if model is metamorphically fair to sentence
	'''
	mutations = perturbator(x, attribute)
	real_predicted = get_model_output_class(model, tokenizer, x)
	# for each mutation, check that output class is the same
	for mutation in mutations:
		mutation_output = get_model_output_class(model, tokenizer, mutation)
		if real_predicted != mutation_output:
			return False, len(mutations)
	return True, len(mutations)

In [ ]:
# get model
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Capstone/models/no_debias/pytorch_model.bin',
                                                      config='/content/drive/MyDrive/NLP Capstone/models/no_debias/config.json')
model = model.to(torch.device('cuda'))
input_embeddings = model.get_input_embeddings()
i = 0
for param in input_embeddings.parameters():
	if i == 0:
		NUM_EMBEDDINGS = int(param.size()[0])
		break
print('NUM_EMBEDDINGS:', NUM_EMBEDDINGS)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

all_word_embeddings = []
for i in range(NUM_EMBEDDINGS):
    token = torch.IntTensor([i]).to(model.device)
    embedding = input_embeddings(token)[0].cpu().detach().numpy()
    all_word_embeddings.append(embedding)
all_word_embeddings = np.array(all_word_embeddings)
print('word embeddings matrix:', all_word_embeddings.shape)

NUM_EMBEDDINGS: 30522



word embeddings matrix: (30522, 768)


In [ ]:
# run bias evaluation
twitter_file = pd.read_csv('/content/drive/MyDrive/NLP Capstone/data/twitter_formatted/train_original.tsv', header=None, sep='\t')
sentences = 0
sentences_w_mutations = 0
violations = 0
for i, row in twitter_file.iterrows():
    if i % 100 == 0:
        print(i, sentences_w_mutations, violations)
    sentences += 1
    is_fair, num_mutations = is_metamorphic_fair(model, tokenizer, row[2], 'gender')
    if num_mutations > 0:
        sentences_w_mutations += 1
    if not is_fair:
        violations += 1
print('final stats:')
print('\tsentences:', sentences)
print('\tsentences with mutations:', sentences_w_mutations)
print('\tviolations:', violations)

0 0 0
10 3 0
20 6 0
30 10 0
40 16 0
50 21 1
60 22 1
70 25 1
80 28 1
90 33 1
100 35 1
110 39 1
120 45 1
130 50 1


KeyboardInterrupt: ignored

Certified Mitigation code

In [ ]:
def certified_mitigation(model, tokenizer, x, mutations, attribute, epsilon):
	'''Return smoothed output using certified mitigation
	'''
	input_pred = get_model_output(model, tokenizer, [x])
	k = len(mutations)
	smoothed_output = input_pred * (math.exp(epsilon) / (k + math.exp(epsilon)))
	# for each mutation, calculate and add to smoothed output
	for mutation in mutations:
		mutation_output = get_model_output(model, tokenizer, [mutation])
		smoothed_output += mutation_output * (1 / (k + math.exp(epsilon)))
	# return final smoothed output class
	return int(np.argmax(smoothed_output.cpu()))

def epsilon_k_fairness(model, tokenizer, x, attribute, epsilon):
	"""Measures epsilon-k fairness given an epsilon value. Epsilon denotes how much
	weight we want to put onto the original sentence while each perturbation is
	given equal weight.

	Args:
		model: An instance of PyTorch torch.nn.Module.
		tokenizer: PreTrainedTokenizer.
		x: the input sentence
		epsilon: flexibility to degree of fairness

	Returns:
		The fairness measured as the difference between the real measured output
		smoothed epsilon-k output
	"""
	before_violations = 0
	after_violations = 0
	mutations = perturbator(x, attribute) # bottleneck of performance in this system

	# find number of fairness violations WITHOUT certified mitigation
	real_predicted = get_model_output_class(model, tokenizer, x)
	for mutation in mutations:
		mutation_output = get_model_output_class(model, tokenizer, mutation)
		if real_predicted != mutation_output:
			before_violations += 1

	# find number of fairness violations WITH certified mitigation
	input_cm_output = certified_mitigation(model, tokenizer, mutations, x, epsilon)
	for mutation in mutations:
		mutation_output = certified_mitigation(model, tokenizer, mutation, mutations, epsilon)
		if input_cm_output != mutation_output:
			after_violations += 1

	# return percentage of violations after and before certified mitigation
	return before_violations, after_violations

In [ ]:
EPSILON = 0.1
ATTRIBUTE = 'gender'